In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Airplane ML") \
    .getOrCreate()

In [2]:
flights = spark.read.parquet('/data/parquet/flights')

In [15]:
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F

flights = flights.na.drop() \
    .withColumn('Delayed', (flights.ArrDelay > 20).cast('string')) \
    .withColumn('Year', flights.Year.cast('integer')) \
    .withColumn('Month', flights.Month.cast('integer')) \
    .withColumn('DayofMonth', flights.DayofMonth.cast('integer')) \
    .withColumn('DayOfWeek', flights.DayOfWeek.cast('integer')) \
    .withColumn('DepTime', flights.DepTime.cast('integer'))

In [16]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

(trainingData, testData) = flights.randomSplit([0.7, 0.3])

labelIndexer = StringIndexer(inputCol='Delayed', outputCol='DelayedIndex').fit(trainingData)
categoricalIndexers = [ StringIndexer(inputCol=inputColName, outputCol=inputColName + 'Index').fit(trainingData) \
                       for inputColName in ['UniqueCarrier', 'FlightNum', 'Origin', 'Dest'] ]

assembler = VectorAssembler(
    inputCols=['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrierIndex', 'FlightNumIndex', 'OriginIndex', 'DestIndex'],
    outputCol='features')

rf = RandomForestClassifier(labelCol="DelayedIndex", featuresCol="features")

pipeline = Pipeline(stages=[labelIndexer, *categoricalIndexers, assembler, rf])

model = pipeline.fit(trainingData)

Py4JJavaError: An error occurred while calling o427.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 24.0 failed 1 times, most recent failure: Lost task 3.0 in stage 24.0 (TID 93, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:377)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.util.Utils$.getIteratorSize(Utils.scala:1760)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala

In [11]:
categoricalIndexers = [ StringIndexer(inputCol=inputColName, outputCol=inputColName + 'Index').fit(trainingData) \
                       for inputColName in ['UniqueCarrier', 'FlightNum', 'Origin', 'Dest'] ]